In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import bias_functions
import text_extraction
import asyncio

In [84]:
#df = pd.read_csv('extracted_data.txt', names=['Name', 'Age', 'City', 'Gender', 'UG_College', 'PG_College', 'Degree', 'Current_Employer', 'Experience(in Yrs)'])

In [85]:
df = bias_functions.read_json_files('./uploads/parsed_json/')

In [86]:
df.head()

,gender,degree,institute,year,city,employer,experience,experience_range,keywords,age
0,NaN,Bachelor of Technology,UPTU,2002,Bareilly,Avida Technologies,0,GREATER THAN 10 YEAR,"[Sensitive, Functional Requirements Specificat...",0.0
1,Male,Bachelor of Engineering,College of Engineering and Technology,1993,Chennai,Mercantiler InfoTech pvt ltd,0,8-10 YEAR,"[Microsoft Visual Studio, User Experience Desi...",0.0
2,Male,Bachelor of Engineering,Bharathiyar University,1982,Bangalore,infinite computer solution,0,GREATER THAN 10 YEAR,"[Human Resources Management System, Hospital M...",1.0
3,NaN,Bachelor of Technology,Jawaharlal Nehru technological University,NaN,NaN,for Five,0,3-5 YEAR,"[Confident, Manhattan Warehouse Management Sys...",NaN
4,NaN,Master of Computer Applications,Telangana University,NaN,NaN,Cybage Software Pvt Ltd,0,GREATER THAN 10 YEAR,"[Hard Working, Independent, Mirroring, Potenti...",NaN


In [87]:
df.shape

(108, 10)

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            33 non-null     object 
 1   degree            97 non-null     object 
 2   institute         92 non-null     object 
 3   year              70 non-null     object 
 4   city              72 non-null     object 
 5   employer          102 non-null    object 
 6   experience        108 non-null    object 
 7   experience_range  108 non-null    object 
 8   keywords          108 non-null    object 
 9   age               70 non-null     float64
dtypes: float64(1), object(9)
memory usage: 8.6+ KB


In [89]:
def process_column(column):
    def process_value(value):
        if isinstance(value, str):
            value = value.strip()
            match = re.match(r'(\d+(\.\d+)?)\+', value)
            if match:
                return float(match.group(1))
            match = re.match(r'(\d+(\.\d+)?)\s*years?', value, re.IGNORECASE)
            if match:
                return float(match.group(1))
            if value.upper() == 'NULL':
                return np.nan
            try:
                return float(value)
            except ValueError:
                return np.nan  
        elif isinstance(value, (int, float)):
            return float(value)
        else:
            return np.nan  
    return column.apply(process_value)

In [91]:
df['experience'] = process_column(df['experience'])

In [92]:
def clean_column(column):
    def clean_value(value):
        if isinstance(value, str):
            value = value.strip()
            value = value.replace('"', '')
            if any(char.isdigit() for char in value):
                return np.nan
            if value == 'NULL' or value == 'India' or value == 'Gender' or value == 'City':
                return np.nan
        return value
    return column.apply(clean_value)

In [93]:
df['gender'] = clean_column(df['gender'])

In [94]:
def clean_and_convert_column(column):
    def clean_value(value):
        if pd.isna(value): 
            return value
        value = str(value).strip() 
        try:
            return float(value)
        except ValueError:
            if any(char.isdigit() for char in value):
                try:
                    return float(value)
                except ValueError:
                    return np.nan
            else:
                return np.nan
    return column.apply(clean_value)

In [95]:
df['age'] = clean_and_convert_column(df['age'])

In [96]:
df['city'] = clean_column(df['city'])

In [98]:
df['employer'] = clean_column(df['employer'])
df['degree'] = clean_column(df['degree'])
df['institute'] = clean_column(df['institute'])

In [99]:
df.head()

,gender,degree,institute,year,city,employer,experience,experience_range,keywords,age
0,NaN,Bachelor of Technology,UPTU,2002,Bareilly,Avida Technologies,0.0,GREATER THAN 10 YEAR,"[Sensitive, Functional Requirements Specificat...",0.0
1,Male,Bachelor of Engineering,College of Engineering and Technology,1993,Chennai,Mercantiler InfoTech pvt ltd,0.0,8-10 YEAR,"[Microsoft Visual Studio, User Experience Desi...",0.0
2,Male,Bachelor of Engineering,Bharathiyar University,1982,Bangalore,infinite computer solution,0.0,GREATER THAN 10 YEAR,"[Human Resources Management System, Hospital M...",1.0
3,NaN,Bachelor of Technology,Jawaharlal Nehru technological University,NaN,NaN,for Five,0.0,3-5 YEAR,"[Confident, Manhattan Warehouse Management Sys...",NaN
4,NaN,Master of Computer Applications,Telangana University,NaN,NaN,Cybage Software Pvt Ltd,0.0,GREATER THAN 10 YEAR,"[Hard Working, Independent, Mirroring, Potenti...",NaN


In [100]:
jd = """Landis+Gyr is a leading global provider of integrated energy management solutions. We measure and analyze energy utilization to generate empowering analytics for smart grid and infrastructure management, enabling utilities and consumers to reduce energy consumption. Our innovative and proven portfolio of software, services and intelligent sensor technology is a key driver to decarbonize the grid. With sales of USD 1.7 billion in FY 2022, Landis+Gyr employs over 7,000 talented people across 30 countries and five continents.

For more than a century, we've been pioneers in energy innovation, and we continue to lead the way toward a more sustainable and efficient energy landscape.

Join us at Landis+Gyr, where we manage energy better - since 1896.

Key Purpose of Role:

This role has responsiblity for development and expected to work in all areas of software engineering ( design, implement, review, document, test, support) as the demand dictates.

Experience Required: 3-10 years

Areas of Responsibility / Tasks:
• To design And Develop Applications software, APIs and enhancement in existing functionality.
• Involvement In Software Development Life-Cycle Activities (requirement gathering, elaboration, solicit clarification, WBS, Estimations, documentation).
• To develop specifications including Functional Requirements, Use Cases and Business Rules, Detailed Design, Code, And Test Procedures As Part Of Agile/Iterative Project Team.
• Document, Train And Support Operations And Software Quality Assurance TeamsPerform Code Reviews
• To perform unit/Integration testing
• Database Design and Maintenance.
• To Setup/ Configure/ Troubleshoot test environment
• Identify the quality KPI’s and ensure the compliance at each stage.
• Object-Oriented programming skills, Strong in .NET concepts
• Technology proficiency, C# with Windows Forms, WPF
• Database (Oracle/ SQL Server/My SQL/MS Access)
• Experience With Common Design Patterns,Documentation
• Experience using Defect management, Source Code Managenent, REQUIREMENT Management tools (Rational/TFS/Devops/Other...)
• Serial Communication/Device Communication (RS232/RS485/RF/GSM/GPRS)/DLMS

We are thrilled that you’ve considered Landis+Gyr to be a part of your professional journey.  As a part of the Landis+Gyr family, employees are offered a comprehensive and competitive package of benefits including medical, dental and vision coverage, life insurance, and a 401(k)-retirement plan featuring a generous company match incentive.  Additionally, we offer three (3) weeks of Paid Time Off (PTO) as well as eleven (11) paid holidays.  We also offer a tuition reimbursement program, optional pet insurance, supplemental medical coverage and a host of other perks to ensure the health, wellness, and enrichment of our team members. 

Pay is in the range of XY per year. The position is eligible for a 401(k) match and consideration for an annual bonus. The actual base salary offered depends on a variety of factors, which may include, as applicable, the qualifications of the individual applicant for the position, years of relevant experience, specific and unique skills, level of education attained, and certifications or other professional licenses held.  All candidates are encouraged to apply. 

All qualified applicants will receive consideration for employment without regard to race, color, religion, sex, sexual orientation, gender identity, national origin, disability, or status as a protected veteran. 

Submit your application – it takes less than 5 minutes! We will review and if selected to move forward we will have a 15-30 minute introduction call to discuss mutual expectations. An interview with the hiring manager will be next followed by an interview with key stakeholders.

We stand for a workplace that nurtures growth, fosters connection, and embraces authenticity. We believe that fostering a community that values diversity isn't just a goal; it's a way of life. The true magic happens when a group of unique minds comes together – bring yours and #EnergizeYourCareer"""

In [101]:
url = 'https://dev.api.talentmarx.in/api/v1/ml/similarity/'

In [102]:
def create_info_text(df):
    strings = []
    for _, row in df.iterrows():
        s = ""
        keys = row.keys()
        for i, element in enumerate(row):
            s += keys[i] + ": " + str(element).strip() + "\n"
        strings.append(s)
    return strings

In [103]:
text = create_info_text(df)

In [104]:
df['gender'].unique().tolist()

[nan, 'Male', 'Female', 'MALE']

In [106]:
df['gender'].value_counts()

gender
Male      28
Female     4
MALE       1
Name: count, dtype: int64

In [109]:
df['city'].value_counts()

city
Bangalore         11
Chennai            5
Hyderabad          5
Greater Noida      4
Mumbai             4
Bengaluru          4
Coimbatore         3
Pune               3
Ghaziabad          2
Kannur             2
Noida              2
Kottayam           1
CHENNAI            1
Theni              1
Phagwara           1
Delhi              1
Nagpur             1
Sultanpur          1
Tirunelveli Dt     1
Hyd                1
Karimnagar         1
Karur              1
Madurai            1
Bareilly           1
New Delhi          1
Jaipur             1
Ahmedabad          1
Jamshedpur         1
Patna              1
Gurugram           1
Anantapur          1
kakinada           1
Trichy             1
Erode              1
Dharuhera          1
Udumalpet          1
Tirupur            1
Name: count, dtype: int64

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            33 non-null     object 
 1   degree            97 non-null     object 
 2   institute         92 non-null     object 
 3   year              70 non-null     object 
 4   city              71 non-null     object 
 5   employer          101 non-null    object 
 6   experience        108 non-null    float64
 7   experience_range  108 non-null    object 
 8   keywords          108 non-null    object 
 9   age               70 non-null     float64
dtypes: float64(2), object(8)
memory usage: 8.6+ KB


In [ ]:
def find_score(colname):
  for val in df[colname].unique().tolist():
    if val == np.nan:
      continue
    temp_df = df.copy()
    temp_df[colname] = val
    resume_data = create_info_text(temp_df[['age', 'city', 'gender', 'institute', 'degree',
       'employer', 'experience']])
    data = {
      "queryDocumentString": jd,
      "documentStrings": resume_data
    }
    response = requests.post(url, json=data)
    scores = eval(response.text)['similarities']
    df["Score_{}_{}".format(colname, val)] = scores
  return "Added Columns"

In [ ]:
find_score('gender')

'Added Columns'

In [ ]:
find_score('city')

'Added Columns'

In [ ]:
for val in ['Hindu', 'Muslim', 'Sikh', 'Chsristian', 'Jain', 'Budhh', 'Jew']:
    if val == np.nan:
      continue
    temp_df = df.copy()
    temp_df['Religion'] = val
    resume_data = create_info_text(temp_df[['age', 'city', 'gender', 'institute', 'degree',
       'employer', 'experience', 'Religion']])
    data = {
      "queryDocumentString": jd,
      "documentStrings": resume_data
    }
    response = requests.post(url, json=data)
    scores = eval(response.text)['similarities']
    df["Score_{}_{}".format('Religion', val)] = scores

In [ ]:
for val in ['B.Tech', 'B.E', 'B.Com', 'B.A', 'B.B.A', 'B.Sc', 'M.Tech', 'M.Sc', 'M.B.A', 'M.Com']:
    if val == np.nan:
      continue
    temp_df = df.copy()
    temp_df['Degree'] = val
    resume_data = create_info_text(temp_df[['age', 'city', 'gender', 'institute', 'degree',
       'employer', 'experience']])
    data = {
      "queryDocumentString": jd,
      "documentStrings": resume_data
    }
    response = requests.post(url, json=data)
    scores = eval(response.text)['similarities']
    df["Score_{}_{}".format('Degree', val)] = scores

In [ ]:
df.head()

,Name,Age,City,Gender,UG_College,PG_College,Degree,Current_Employer,Experience(in Yrs),Score_Gender_nan,...,Score_Degree_B.Tech,Score_Degree_B.E,Score_Degree_B.Com,Score_Degree_B.A,Score_Degree_B.B.A,Score_Degree_B.Sc,Score_Degree_M.Tech,Score_Degree_M.Sc,Score_Degree_M.B.A,Score_Degree_M.Com
0,Name,NaN,NaN,NaN,UG,PG,Degree,Employer,NaN,0.146558,...,0.161710,0.168281,0.155715,0.180936,0.198672,0.189304,0.165097,0.190847,0.201857,0.153503
1,A.DHANALAKSHMI,NaN,Hyderabad,NaN,J.N.T.U.A,NaN,B.Tech. (ECE),Magus IT Solutions Pvt Ltd,2.1,0.243285,...,0.241963,0.250051,0.229652,0.256853,0.252451,0.255897,0.244072,0.259243,0.254447,0.231925
2,AliKhan Makhool,36.0,Bareilly,Male,UPTU,NaN,B.Tech. (CSE),Avida Technologies,4.3,0.154305,...,0.150311,0.154697,0.135621,0.154023,0.150683,0.156417,0.152512,0.157886,0.151985,0.135642
3,Anil,NaN,NaN,NaN,Male,Baptla Engg College,M.Tech.,Minacs Marketing Services,3.0,0.171199,...,0.171523,0.149156,0.117832,0.153897,0.150442,0.157153,0.173374,0.157967,0.150796,0.118936
4,Arun Kumar S,31.0,Chennai,Male,SBM College of Engineering and Technology,NaN,B.E. (ECE),Mercantiler InfoTech Pvt Ltd,1.5,0.179238,...,0.188923,0.188086,0.180724,0.186280,0.189013,0.193530,0.191706,0.196154,0.188941,0.183114


In [ ]:
df.describe()

,Age,Experience(in Yrs),Score_Gender_nan,Score_Gender_Male,Score_Gender_Female,Score_City_nan,Score_City_Hyderabad,Score_City_Bareilly,Score_City_Chennai,Score_City_Bangalore,...,Score_Degree_B.Tech,Score_Degree_B.E,Score_Degree_B.Com,Score_Degree_B.A,Score_Degree_B.B.A,Score_Degree_B.Sc,Score_Degree_M.Tech,Score_Degree_M.Sc,Score_Degree_M.B.A,Score_Degree_M.Com
count,90.000000,118.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,...,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000
mean,34.635556,8.857263,0.206404,0.211637,0.211932,0.209952,0.208889,0.224215,0.207851,0.205631,...,0.208669,0.211272,0.197947,0.212704,0.213342,0.215243,0.209837,0.216302,0.212896,0.197956
std,5.348828,3.550996,0.045175,0.044016,0.044190,0.047935,0.046899,0.046594,0.048205,0.046738,...,0.043974,0.045999,0.046514,0.046358,0.045572,0.045792,0.043779,0.045913,0.045514,0.046719
min,11.000000,1.100000,0.021462,0.041875,0.039159,0.024909,0.021462,0.051064,0.007981,0.026871,...,0.041121,0.021504,0.010838,0.022326,0.024145,0.024172,0.041513,0.023431,0.023468,0.010511
25%,32.000000,6.940000,0.182688,0.187192,0.187104,0.180613,0.182974,0.196774,0.182490,0.178133,...,0.184424,0.189551,0.176498,0.186761,0.188158,0.193240,0.186608,0.195339,0.187665,0.177268
50%,34.000000,8.800000,0.208440,0.214127,0.212815,0.209557,0.214385,0.227406,0.210474,0.206994,...,0.214770,0.218018,0.202687,0.221303,0.220397,0.221186,0.214468,0.223173,0.221590,0.204480
75%,37.750000,11.957500,0.236741,0.244283,0.242808,0.242410,0.241136,0.257659,0.240124,0.238503,...,0.234954,0.238453,0.228184,0.241540,0.239059,0.240659,0.237179,0.244772,0.240584,0.227884
max,45.000000,18.000000,0.301214,0.309299,0.307517,0.311469,0.319963,0.316839,0.316497,0.309299,...,0.307122,0.318644,0.307364,0.326037,0.331066,0.320029,0.309770,0.322910,0.333295,0.309419


In [ ]:
df.columns

Index(['Name', 'Age', 'City', 'Gender', 'UG_College', 'PG_College', 'Degree',
       'Current_Employer', 'Experience(in Yrs)', 'Score_Gender_nan',
       'Score_Gender_Male', 'Score_Gender_Female', 'Score_City_nan',
       'Score_City_Hyderabad', 'Score_City_Bareilly', 'Score_City_Chennai',
       'Score_City_Bangalore', 'Score_City_Coimbatore', 'Score_City_Gurgaon',
       'Score_City_Bhavani', 'Score_City_Secunderabad',
       'Score_City_Puttaparthi', 'Score_City_Ghaziabad', 'Score_City_Gurugram',
       'Score_City_Kolkata', 'Score_City_Noida', 'Score_City_Navi Mumbai',
       'Score_City_Haridwar', 'Score_City_Patna', 'Score_City_Bengaluru',
       'Score_City_Pune', 'Score_City_Ahmedabad', 'Score_City_New Delhi',
       'Score_City_Jaipur', 'Score_City_Visakhapatnam', 'Score_City_Kannur',
       'Score_City_Mumbai', 'Score_City_Greater Noida', 'Score_City_Nagpur',
       'Score_City_Phagwara', 'Score_City_Theni', 'Score_City_Kottayam',
       'Score_City_NCR Delhi', 'Score_City_S

In [ ]:
df[['Score_City_Bangalore', 'Score_City_Greater Noida']]

,Score_City_Bangalore,Score_City_Greater Noida
0,0.152415,0.180499
1,0.245043,0.277541
2,0.137121,0.156278
3,0.155495,0.174377
4,0.184534,0.196034
...,...,...
120,0.174844,0.197103
121,0.167188,0.179974
122,0.251334,0.241126
123,0.161055,0.170376
